In [1]:
# !pip install git+https://github.com/openai/whisper.git

In [2]:
import os
import time
import pandas as pd
import torch
from transformers import pipeline

2024-10-24 08:43:42.592831: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-24 08:43:42.592965: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-24 08:43:42.729923: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# all files without 6 and 7 (they are too long)
files_directory = '/kaggle/input/speech-to-text/'

audio_files_for_transcribtion = [file for file in os.listdir(files_directory)
                                 if file.startswith('sample') and 'out' not in file ]

audio_files_for_transcribtion.remove('sample4.mp3')
audio_files_for_transcribtion.remove('sample7.mp3')

audio_files_for_transcribtion.sort( key=lambda x:int("".join(filter(str.isdigit,x))))
audio_files_for_transcribtion

['sample1.mp3',
 'sample2.mp3',
 'sample3.mp3',
 'sample8.mp3',
 'sample9.mp3',
 'sample10.mp3',
 'sample11.mp3',
 'sample12.mp3',
 'sample13.mp3',
 'sample14.mp3',
 'sample15.mp3']

# ivrit

In [5]:
#model_nm = 'ivrit-ai/whisper-13-v2-e3'
model_nm = "ivrit-ai/whisper-large-v2-tuned"

In [24]:


device = "cuda:0" if torch.cuda.is_available() else "cpu"

pipe = pipeline(
  "automatic-speech-recognition",
  model= model_nm,
  chunk_length_s=30,
  batch_size=8,
  device=device,
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [25]:
audio_file_path = '/kaggle/input/speech-to-text/sample8.mp3'

In [26]:
%%time
prediction = pipe(audio_file_path, generate_kwargs={"language": "hebrew"})["text"]

CPU times: user 8.82 s, sys: 15.5 ms, total: 8.83 s
Wall time: 8.94 s


In [10]:
prediction

'הלו, עוזי? כן? חברת טוט, מה קורה? בסדר יופי, מתקשרת לגבי הצעות מחיר איפה אתה נמצא היום באינטה ובביתי? בשבילכם בסלולר לא, לא רלוונטי, תודה למה לא רלוונטי? לא רלוונטי, לא רלוונטי, תודה מה לא רלוונטי? אין לך סלולר? לא רלוונטי, נשמה, לא רלוונטי מה זה לא רלוונטי? אין לך סלולר? אני חייבת להבין מה זה התשובה הזאתי שלך מה, את רצינית איתי? מה זה לא רלוונטי, אין לך סלולר... אני אומר לך שלא רלוונטי מה שאת מציעה לי לא רלוונטי, חינם, חינם, לא רלוונטי אבל עוד לא שמעת אותי אבל הנה, תגיד לי, חינם, קח טלפון חינם, אל תגיד לי לא רלוונטי אני אגיד לך חינם, אין דבר כזה חינם אז זה לא רלוונטי אני אומר לך, מה את רווייתי? אבל עוד לא שמעת, זה חטא'

In [10]:
d = []
for i,f in enumerate(audio_files_for_transcribtion):
    print(i, end='')
    start = time.time()
    text = pipe(files_directory+f, generate_kwargs={"language": "hebrew"})["text"]
    end = time.time()
    d.append(
        {
            'name':f,
            'text':text,
            'run_time':end - start,
            'engine':torch.cuda.get_device_name()
  
                    }
    )

results = pd.DataFrame(d)
results

0123456789

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


10

,name,text,run_time,engine
0,sample1.mp3,"הלו שלום היי שלום, שנייה הלו שלום הלו, מי אני ...",16.123071,Tesla T4
1,sample2.mp3,"היי רותי, רועי מראל, מה הביטוח שלכם, מה שלומך?...",12.014237,Tesla T4
2,sample3.mp3,"שלום, ותודה שתניתם לכולם. מקומכם בתור הינו, אח...",20.412798,Tesla T4
3,sample8.mp3,"הלו, עוזי? כן? חברת טוט, מה קורה? בסדר יופי, מ...",8.944068,Tesla T4
4,sample9.mp3,"הלו? שלום. כן, שלום. אני מפרק, מה אתה רואה? אה...",6.888821,Tesla T4
5,sample10.mp3,"אבל כן, אבל המהירות גלישה שלנו הרבה יותר גבוהה...",21.519279,Tesla T4
6,sample11.mp3,"שלום אלון, נכון, שלום לך, מדבר את ורד מאיחוד מ...",15.400238,Tesla T4
7,sample12.mp3,"פלקום שלום, תודה על ההמתנה. בוקר טוב. בוקר מצו...",27.975692,Tesla T4
8,sample13.mp3,"בסדרוהחלפת נורא שנשרפה בעלות הנורא בלבד. כן, פ...",9.820299,Tesla T4
9,sample14.mp3,"הלו? כן, שלום. שלום, אני מתקשרת בקשר למודעה שר...",26.842724,Tesla T4


In [11]:
results['text_length'] = results['text'].str.len()
results['model'] = model_nm
results

,name,text,run_time,engine,text_length,model
0,sample1.mp3,"הלו שלום היי שלום, שנייה הלו שלום הלו, מי אני ...",16.123071,Tesla T4,812,ivrit-ai/whisper-large-v2-tuned
1,sample2.mp3,"היי רותי, רועי מראל, מה הביטוח שלכם, מה שלומך?...",12.014237,Tesla T4,1047,ivrit-ai/whisper-large-v2-tuned
2,sample3.mp3,"שלום, ותודה שתניתם לכולם. מקומכם בתור הינו, אח...",20.412798,Tesla T4,1296,ivrit-ai/whisper-large-v2-tuned
3,sample8.mp3,"הלו, עוזי? כן? חברת טוט, מה קורה? בסדר יופי, מ...",8.944068,Tesla T4,626,ivrit-ai/whisper-large-v2-tuned
4,sample9.mp3,"הלו? שלום. כן, שלום. אני מפרק, מה אתה רואה? אה...",6.888821,Tesla T4,422,ivrit-ai/whisper-large-v2-tuned
5,sample10.mp3,"אבל כן, אבל המהירות גלישה שלנו הרבה יותר גבוהה...",21.519279,Tesla T4,1548,ivrit-ai/whisper-large-v2-tuned
6,sample11.mp3,"שלום אלון, נכון, שלום לך, מדבר את ורד מאיחוד מ...",15.400238,Tesla T4,1113,ivrit-ai/whisper-large-v2-tuned
7,sample12.mp3,"פלקום שלום, תודה על ההמתנה. בוקר טוב. בוקר מצו...",27.975692,Tesla T4,1888,ivrit-ai/whisper-large-v2-tuned
8,sample13.mp3,"בסדרוהחלפת נורא שנשרפה בעלות הנורא בלבד. כן, פ...",9.820299,Tesla T4,502,ivrit-ai/whisper-large-v2-tuned
9,sample14.mp3,"הלו? כן, שלום. שלום, אני מתקשרת בקשר למודעה שר...",26.842724,Tesla T4,2115,ivrit-ai/whisper-large-v2-tuned


In [15]:
results.to_excel('whisper_Transcription.xlsx')

# faster whisper

In [28]:
! pip install ffmpeg
! pip install faster_whisper
import ffmpeg
import faster_whisper
import time
import pandas as pd

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6080 sha256=f26cf87ec4549232f31b5c149802b752d000faa2a13f1a404af834dea60a06e7
  Stored in directory: /root/.cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffmpeg
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 41.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 49.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 38.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 34.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.5 MB/s eta 0:00:00


In [29]:
# Initialize the model
model_nm = 'ivrit-ai/faster-whisper-v2-pd1-e1'
model = faster_whisper.WhisperModel(model_nm)

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.80k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

vocabulary.json:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

RuntimeError: CUDA failed with error out of memory

In [ ]:
def transcribe_file(file_name):
    start = time.time()
    
    segs, _ = model.transcribe(file_name, language='he')
    l = []
    for seg in segs:
        l.append(seg.text)
    end = time.time()
    run_time = end - start
    
    return "".join(l),run_time

In [ ]:
device_name = torch.cuda.get_device_name()

In [ ]:
d = []
for i,f in enumerate(audio_files_for_transcribtion):
    print(i, end='')
    text,run_time = transcribe_file(files_directory+f)
    d.append(
        {
            'name':f,
            'text':text,
            'run_time':run_time,
            'engine': device_name
  
                    }
    )

results = pd.DataFrame(d)
results

In [ ]:
results['text_length'] = results['text'].str.len()
results['model'] = model_nm
results

In [ ]:
results.to_excel('faster_whisper_Transcription.xlsx')